In [4]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spy
from tqdm import tqdm
import time
import pickle
import prince
import mca

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

# 진단(diagnoses) 파일 로드 및 value_counts

In [5]:
## 질병진단 테이블
diag = pd.read_table('./data/DIAGNOSES_ICD.csv', sep=',')
diag.dropna(inplace=True)

diag_stat = diag['ICD9_CODE'].value_counts().to_frame().reset_index()
diag_stat.columns = ['ICD9_CODE', 'count']

## ICD9 to 진단명
d_icd = pd.read_table('./data/D_ICD_DIAGNOSES.csv', sep=',')
d_icd = d_icd[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']]

## diag에 진단명 merge
diag_title = pd.merge(diag_stat, d_icd, on='ICD9_CODE', how='left').head(30)
diag_title

,ICD9_CODE,count,SHORT_TITLE,LONG_TITLE
0,4019,20703,Hypertension NOS,Unspecified essential hypertension
1,4280,13111,CHF NOS,"Congestive heart failure, unspecified"
2,42731,12891,Atrial fibrillation,Atrial fibrillation
3,41401,12429,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...
4,5849,9119,Acute kidney failure NOS,"Acute kidney failure, unspecified"
5,25000,9058,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...
6,2724,8690,Hyperlipidemia NEC/NOS,Other and unspecified hyperlipidemia
7,51881,7497,Acute respiratry failure,Acute respiratory failure
8,5990,6555,Urin tract infection NOS,"Urinary tract infection, site not specified"
9,53081,6326,Esophageal reflux,Esophageal reflux


# 처음 질병 (FIRST_ICD), 나중 질병 (SECND_ICD) 설정 후, 각 hadm_id에 부여

In [6]:
FIRST_ICD = '4019'
SECND_ICD = '4280'


def func1(x):
    if x == FIRST_ICD:
        return 'F'
    if x == SECND_ICD:
        return 'S'
    return 'N'


diag['FSX'] = diag['ICD9_CODE'].map(func1)
diag['FSX'].value_counts()
diag

N    617186
F     20703
S     13111
Name: FSX, dtype: int64

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,FSX
0,1297,109,172335,1.0,40301,N
1,1298,109,172335,2.0,486,N
2,1299,109,172335,3.0,58281,N
3,1300,109,172335,4.0,5855,N
4,1301,109,172335,5.0,4254,N
...,...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280,N
651043,639799,97503,188195,3.0,V5869,N
651044,639800,97503,188195,4.0,V1279,N
651045,639801,97503,188195,5.0,5275,N


# hadm_id 별로 진단 질병 부여 
* (F(first), S(second), X(both), np.nan(해당없음))
* 하나의 hadm_id에 여러 질병 진단 가능

In [8]:
def func2(x):
    if ('F' in x.values) and ('S' in x.values):
        return 'X'
    if ('F' in x.values):
        return 'F'
    if ('S' in x.values):
        return 'S'
    return np.nan


FSX_info = diag.groupby(['SUBJECT_ID', 'HADM_ID'
                         ])['FSX'].apply(func2).reset_index().dropna()
FSX_info['FSX'].value_counts()
FSX_info

F    15940
S     8355
X     4756
Name: FSX, dtype: int64

,SUBJECT_ID,HADM_ID,FSX
1,3,145834,S
7,9,150750,X
10,12,112213,F
11,13,143045,F
15,18,188822,F
...,...,...,...
58923,99983,117390,F
58925,99991,151118,S
58926,99992,197084,F
58927,99995,137810,S


# 입원 (admission) 시각 불러와서 merge 

In [9]:
adms = pd.read_table('./data/ADMISSIONS.csv',
                     sep=',')[['HADM_ID', 'ADMITTIME']]

FSX_info = pd.merge(FSX_info, adms, how='left', on='HADM_ID')
FSX_info

,SUBJECT_ID,HADM_ID,FSX,ADMITTIME
0,3,145834,S,2101-10-20 19:08:00
1,9,150750,X,2149-11-09 13:06:00
2,12,112213,F,2104-08-07 10:15:00
3,13,143045,F,2167-01-08 18:43:00
4,18,188822,F,2167-10-02 11:18:00
...,...,...,...,...
29046,99983,117390,F,2193-04-26 11:35:00
29047,99991,151118,S,2184-12-24 08:30:00
29048,99992,197084,F,2144-07-25 18:03:00
29049,99995,137810,S,2147-02-08 08:00:00


# first-second disease진단 hadm 그룹과 first disease only hadm그룹 추출

In [10]:
def func4(df1):

    df1['ADMITTIME'] = pd.to_datetime(df1['ADMITTIME'])
    df2 = df1.sort_values(by='ADMITTIME')

    FSX_set = df2['FSX'].unique()
    if len(FSX_set) == 1:
        if FSX_set[0] == 'F':
            F_hadm = df2.iloc[0]['HADM_ID']
            return pd.Series([F_hadm, 'N/A', 'N/A'])
        else:
            return pd.Series([np.nan, np.nan, np.nan])

    for ii in range(len(df2))[:-1]:
        if df2.iloc[ii]['FSX'] == 'X':
            return pd.Series([np.nan, np.nan, np.nan])

        if df2.iloc[ii]['FSX'] == 'S':
            return pd.Series([np.nan, np.nan, np.nan])

        if (df2.iloc[ii]['FSX'] == 'F') and (df2.iloc[ii + 1]['FSX']
                                             in ['S', 'X']):
            F_hadm = df2.iloc[ii]['HADM_ID']
            S_hadm = df2.iloc[ii + 1]['HADM_ID']
            interval = df2.iloc[ii +
                                1]['ADMITTIME'] - df2.iloc[ii]['ADMITTIME']
            return pd.Series([F_hadm, S_hadm, interval])

    return pd.Series([np.nan, np.nan, np.nan])


FS_mat = FSX_info.groupby('SUBJECT_ID')[['HADM_ID', 'FSX',
                                         'ADMITTIME']].apply(func4)
FS_mat.columns = ['F_hadm', 'S_hadm', 'interval']

FS_mat = FS_mat.dropna(how='all')
FwithS_mat = FS_mat.loc[FS_mat['S_hadm'] != 'N/A']
FwoutS_mat = FS_mat.loc[FS_mat['S_hadm'] == 'N/A']
FwithS_mat
FwoutS_mat

,F_hadm,S_hadm,interval
SUBJECT_ID,,,
111,192123.0,155897,798 days 21:17:00
124,112906.0,138376,1475 days 15:16:00
177,196896.0,143120,355 days 23:46:00
211,193975.0,101148,645 days 11:06:00
236,151459.0,191151,1598 days 13:07:00
...,...,...,...
98050,103030.0,110692,233 days 22:41:00
98593,145414.0,176136,20 days 19:19:00
98744,186076.0,153703,239 days 05:28:00


,F_hadm,S_hadm,interval
SUBJECT_ID,,,
12,112213.0,N/A,N/A
13,143045.0,N/A,N/A
18,188822.0,N/A,N/A
19,109235.0,N/A,N/A
20,157681.0,N/A,N/A
...,...,...,...
99965,101083.0,N/A,N/A
99966,167228.0,N/A,N/A
99983,117390.0,N/A,N/A


# 해당 subject가 주어진 hadm 이전에 진단받은 질병 리스트 제작

* 1) 해당 subject의 성별과 입원 당시 나이
* 2) item(lab) data (binary, abnormal vs normal) <-- 한 item을 여러번 측정시 한번이라도 abnormal이면 1
* 3) 해당 subject가 주어진 hadm 이전에 진단받은 질병 리스트
* 를 return 하는 함수 제작

## pats, labs 데이터 불러오기

In [11]:
pats = pd.read_table('./data/PATIENTS.csv',
                     sep=',')[['SUBJECT_ID', 'GENDER', 'DOB']]
labs = pd.read_table('./data/LABEVENTS_5cols.csv', sep=',')

labs.head()
adms.head()
pats.head()

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM,FLAG
0,3,145834.0,50868,17.0,NaN
1,3,145834.0,50882,25.0,NaN
2,3,145834.0,50893,8.2,abnormal
3,3,145834.0,50902,99.0,abnormal
4,3,145834.0,50910,48.0,NaN


,HADM_ID,ADMITTIME
0,165315,2196-04-09 12:26:00
1,152223,2153-09-03 07:15:00
2,124321,2157-10-18 19:34:00
3,161859,2139-06-06 16:14:00
4,129635,2160-11-02 02:06:00


,SUBJECT_ID,GENDER,DOB
0,249,F,2075-03-13 00:00:00
1,250,F,2164-12-27 00:00:00
2,251,M,2090-03-15 00:00:00
3,252,M,2078-03-06 00:00:00
4,253,F,2089-11-26 00:00:00


## 입원 당시 나이 추출

In [12]:
lab_all = pd.merge(labs, adms, left_on='HADM_ID', right_on='HADM_ID')
lab_all = pd.merge(lab_all, pats, left_on='SUBJECT_ID', right_on='SUBJECT_ID')

lab_all['ADMITTIME'] = pd.to_datetime(lab_all['ADMITTIME'])
lab_all['DOB'] = pd.to_datetime(lab_all['DOB'])
days_diff = (lab_all['ADMITTIME'].dt.date - lab_all['DOB'].dt.date)
lab_all['age'] = days_diff.map(lambda x: x.days // 365)

lab_all.drop(['DOB'], axis=1, inplace=True)
lab_all

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM,FLAG,ADMITTIME,GENDER,age
0,3,145834.0,50868,17.00,NaN,2101-10-20 19:08:00,M,76
1,3,145834.0,50882,25.00,NaN,2101-10-20 19:08:00,M,76
2,3,145834.0,50893,8.20,abnormal,2101-10-20 19:08:00,M,76
3,3,145834.0,50902,99.00,abnormal,2101-10-20 19:08:00,M,76
4,3,145834.0,50910,48.00,NaN,2101-10-20 19:08:00,M,76
...,...,...,...,...,...,...,...,...
20115309,98982,167146.0,51250,85.00,NaN,2145-12-03 17:25:00,F,79
20115310,98982,167146.0,51265,232.00,NaN,2145-12-03 17:25:00,F,79
20115311,98982,167146.0,51277,13.70,NaN,2145-12-03 17:25:00,F,79
20115312,98982,167146.0,51279,4.02,abnormal,2145-12-03 17:25:00,F,79


## 질병 리스트 제작

In [13]:
def generate_status_mat(hadm_ids):
    df = pd.DataFrame(hadm_ids['F_hadm'].unique(), columns=['HADM_ID'])

    for ha in tqdm(hadm_ids['F_hadm']):
        sub_all = lab_all[lab_all['HADM_ID'] == ha]

        for i in sub_all['ITEMID']:
            sub_item = sub_all[sub_all['ITEMID'] == i]

            if 'abnormal' in sub_item['FLAG'].values:
                df.loc[df['HADM_ID'] == ha, 'item : {}'.format(i)] = 1

            else:
                df.loc[df['HADM_ID'] == ha, 'item : {}'.format(i)] = 0

    df_info = pd.merge(df,
                       lab_all[['HADM_ID', 'GENDER', 'age']].drop_duplicates(),
                       how='left')
    col1 = df_info.columns[:1].to_list()
    col2 = df_info.columns[-2:].to_list()
    col3 = df_info.columns[1:-2].to_list()

    new_col = col1 + col2 + col3
    df_info = df_info[new_col]

    df2 = pd.DataFrame()
    for i, sub in tqdm(enumerate(hadm_ids.index)):
        sub_lab = lab_all[lab_all['SUBJECT_ID'] == sub]
        F_hadm = hadm_ids.iloc[i]['F_hadm']

        if F_hadm not in sub_lab['HADM_ID'].values:
            continue

        time = sub_lab[sub_lab['HADM_ID'] == F_hadm]['ADMITTIME'].iloc[0]
        df2 = pd.concat([df2, sub_lab.loc[sub_lab['ADMITTIME'] < time]])

    ex = pd.merge(df_info,
                  lab_all[['HADM_ID', 'SUBJECT_ID']].drop_duplicates(),
                  how='left')
    diag2 = diag.loc[diag['SUBJECT_ID'].isin(ex['SUBJECT_ID'])]

    ex2 = pd.DataFrame(ex['HADM_ID'].unique(), columns=['HADM_ID'])

    for sub in tqdm(ex['SUBJECT_ID']):
        a = diag2[diag2['SUBJECT_ID'] == sub]
        for i, sub2 in enumerate(a['SUBJECT_ID']):
            if sub == sub2:
                ex2.loc[ex['SUBJECT_ID'] == sub,
                        'icd9 : {}'.format(a.iloc[i]['ICD9_CODE'])] = 1
    ex2 = ex2.fillna(0)

    fin_df = pd.merge(df_info, ex2, how='left')

    return fin_df

In [15]:
# df_FwithS_mat = generate_status_mat(FwithS_mat)

  1%|▉                                                                                 | 7/640 [00:01<01:58,  5.34it/s]C:\Users\User\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)
100%|████████████████████████████████████████████████████████████████████████████████| 640/640 [02:00<00:00,  5.33it/s]
640it [00:10, 61.03it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 640/640 [00:08<00:00, 71.60it/s]


In [16]:
# with open('df_FwithS_mat.pickle', 'wb') as f:
#     pickle.dump(df_FwithS_mat, f)

In [17]:
with open('df_FwithS_mat.pickle', 'rb') as f:
    df_FwithS_mat = pickle.load(f)

In [18]:
df_FwithS_mat

,HADM_ID,GENDER,age,item : 51248,item : 51249,item : 51250,item : 51251,item : 51254,item : 51255,item : 51256,...,icd9 : 61179,icd9 : E9399,icd9 : 99939,icd9 : 56731,icd9 : 41404,icd9 : 30183,icd9 : 30928,icd9 : 5933,icd9 : 71697,icd9 : 7469
0,192123.0,F,66.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,112906.0,M,71.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,196896.0,M,76.0,1.0,1.0,1.0,NaN,0.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,193975.0,F,83.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,151459.0,M,52.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,103030.0,F,78.0,1.0,0.0,1.0,NaN,0.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
636,145414.0,F,61.0,0.0,1.0,1.0,NaN,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
637,186076.0,M,69.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
638,165352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# df_FwoutS_mat = generate_status_mat(FwoutS_mat)

  0%|                                                                                | 7/13080 [00:00<22:50,  9.54it/s]C:\Users\User\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)
100%|████████████████████████████████████████████████████████████████████████████| 13080/13080 [37:42<00:00,  5.78it/s]
13080it [04:20, 50.16it/s]
100%|███████████████████████████████████████████████████████████████████████████| 13080/13080 [01:10<00:00, 184.79it/s]


In [20]:
# with open('df_FwoutS_mat.pickle', 'wb') as f:
#     pickle.dump(df_FwoutS_mat, f)

In [21]:
with open('df_FwoutS_mat.pickle', 'rb') as f:
    df_FwoutS_mat = pickle.load(f)

In [22]:
df_FwoutS_mat

,HADM_ID,GENDER,age,item : 51221,item : 51222,item : 51248,item : 51249,item : 51250,item : 51265,item : 51277,...,icd9 : 33721,icd9 : 34691,icd9 : 60781,icd9 : 40591,icd9 : 1838,icd9 : 4374,icd9 : 86122,icd9 : 86113,icd9 : E9654,icd9 : 88013
0,112213.0,M,72.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,143045.0,F,39.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,188822.0,M,50.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109235.0,M,300.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,157681.0,F,75.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13075,101083.0,M,66.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13076,167228.0,M,77.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13077,117390.0,M,78.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13078,197084.0,F,65.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
